In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [5]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


0it [00:00, ?it/s]

9920512it [00:01, 8073831.11it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129559.17it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 1998994.59it/s]                           
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50647.68it/s]            

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)

In [8]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2198
Epoch [1/5], Step [200/600], Loss: 0.2768
Epoch [1/5], Step [300/600], Loss: 0.2369
Epoch [1/5], Step [400/600], Loss: 0.2640
Epoch [1/5], Step [500/600], Loss: 0.2278
Epoch [1/5], Step [600/600], Loss: 0.1356
Epoch [2/5], Step [100/600], Loss: 0.1490
Epoch [2/5], Step [200/600], Loss: 0.0662
Epoch [2/5], Step [300/600], Loss: 0.0601
Epoch [2/5], Step [400/600], Loss: 0.1110
Epoch [2/5], Step [500/600], Loss: 0.0621
Epoch [2/5], Step [600/600], Loss: 0.1091
Epoch [3/5], Step [100/600], Loss: 0.0581
Epoch [3/5], Step [200/600], Loss: 0.0738
Epoch [3/5], Step [300/600], Loss: 0.0549
Epoch [3/5], Step [400/600], Loss: 0.1159
Epoch [3/5], Step [500/600], Loss: 0.0784
Epoch [3/5], Step [600/600], Loss: 0.0137
Epoch [4/5], Step [100/600], Loss: 0.0400
Epoch [4/5], Step [200/600], Loss: 0.0648
Epoch [4/5], Step [300/600], Loss: 0.0699
Epoch [4/5], Step [400/600], Loss: 0.0150
Epoch [4/5], Step [500/600], Loss: 0.0933
Epoch [4/5], Step [600/600], Loss:

In [0]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')